In [1]:
from careerjet_api_client import CareerjetAPIClient
import pandas as pd
%reload_ext sql
%sql mysql://root:@localhost:3306/project_db?charset=utf8
%sql use project_db
import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy import exc
import unicodedata
from datetime import datetime
import us
import time

engine = create_engine('mysql+mysqlconnector://root:@localhost:3306/project_db?charset=utf8', echo=False)

0 rows affected.


/usr/local/lib/python2.7/dist-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/usr/local/lib/python2.7/dist-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [2]:
cj  =  CareerjetAPIClient("en_US");
params = {
    'location'    : '',
    'keywords'    : '',
    'affid'       : '',
    'user_ip'     : '',
    'user_agent'  : 'X-UA-Compatible: Chrome=1',
    'page'        : 61,
    'sort'        : 'date',
    'pagesize'    : '25',
    'url'         : 'http://www.example.com/jobsearch'
}
result_json = cj.search(params)
num_pages = result_json['pages']

In [3]:
first_date = %sql select date from careerjet ORDER BY date DESC limit 1;

1 rows affected.


In [4]:
try:
    fo = open("log_jet.txt", "a")
    fo.write( "\n");
    fo.close()
except:
    pass
!date >> log_jet.txt
try:
    fo = open("log_jet.txt", "a")
    fo.write( "total page number " + str(num_pages) + "\n")
    fo.close()
except:
    pass

In [7]:
cj  =  CareerjetAPIClient("en_US");
p = 0
for s in range(0,51):
    print us.states.STATES[s]
    for i in range(1,50):
        p=p+1
            
        params = {
            'location'    : str(us.states.STATES[s]),
            'keywords'    : '',
            'affid'       : '',
            'user_ip'     : '',
            'user_agent'  : '',
            'page'        : i,
            'sort'        : 'date',
            'pagesize'    : '25',
            'url'         : 'http://www.example.com/jobsearch'
        }

        print i
        result_json = cj.search(params)
        
        if result_json['type']=='JOBS':
            results = result_json['jobs']
            for r in results:
                r['date']=unicodedata.normalize('NFKD', r['date']).encode('ascii','ignore').replace(",","")
                r['date']=datetime.strptime(r['date'],'%a %d %b %Y %X %Z')
                r['date']=r['date'].strftime("%Y-%m-%d %H:%M:%S")

            if len(first_date)==0 or datetime.strptime(results[0]['date'],"%Y-%m-%d %H:%M:%S") > first_date[0][0]:
                df = pd.DataFrame(pd.DataFrame(result_json['jobs']))
                for row in range(0,25):
                    try:
                        df.iloc[row:row+1].to_sql(name='', con=engine, if_exists = 'append', index=False)
                    except exc.IntegrityError as ex:
                        continue
                
            else:                
                try:
                    fo = open("log_jet.txt", "a")
                    fo.write( str(us.states.STATES[s])+ " " + str(i)+ " ")
                    # Close opend file
                    fo.close()
                except:
                    pass
                break
                
            if i == 49:
                try:
                    fo = open("log_jet.txt", "a")
                    fo.write( str(us.states.STATES[s])+ " " + str(i)+ " ")
                    # Close opend file
                    fo.close()
                except:
                    pass
        else:
            try:
                fo = open("log_jet.txt", "a")
                fo.write( " " + str(us.states.STATES[s])+ " " + str(i) + " response error ")
                # Close opend file
                fo.close()
                break
            except:
                break

Alabama
1
Alaska
1
Arizona
1
Arkansas
1
California
1
Colorado
1
Connecticut
1
Delaware
1
District of Columbia
1
Florida
1
Georgia
1
Hawaii
1
Idaho
1
Illinois
1
Indiana
1
Iowa
1
Kansas
1
Kentucky
1
Louisiana
1
Maine
1
Maryland
1
Massachusetts
1
Michigan
1
Minnesota
1
Mississippi
1
Missouri
1
Montana
1
Nebraska
1
Nevada
1
New Hampshire
1
New Jersey
1
New Mexico
1
New York
1
North Carolina
1
North Dakota
1
Ohio
1
Oklahoma
1
Oregon
1
Pennsylvania
1
Rhode Island
1
South Carolina
1
South Dakota
1
Tennessee
1
Texas
1
Utah
1
Vermont
1
Virginia
1
Washington
1
West Virginia
1
Wisconsin
1


In [8]:
try:
    fo = open("log_jet.txt", "a")
    fo.write( "\ntotal pages rolled over:        " + str(p) +"\n");
    # Close opend file
    fo.close()
except:
    pass